In [13]:
# Import required libraries
import pickle
import copy
import pathlib
import dash
import math
import datetime as dt
import pandas as pd
from dash.dependencies import Input, Output, State, ClientsideFunction
import dash_core_components as dcc
import dash_html_components as html
import plotly.graph_objects as go

from jupyter_dash import JupyterDash

In [14]:
sdmx_url = 'https://sdmx.data.unicef.org/ws/public/sdmxapi/rest/data/ECARO,TRANSMONEE,1.0/.{}....?format=csv'

In [15]:
indicators_dict = {
    'EDU_SDG_STU_L2_MATH': {
        'name': 'Proportion of children at the end of lower secondary education reaching minimum proficiency in math', 
        'compare': ['EDU_SDG_STU_L2_READING', 'EDU_SDG_STU_L1_GLAST_MATH', 'EDU_SDG_STU_L1_G2OR3_MATH', 'EDUNF_NERA_L2',]
    },
    'EDU_SDG_STU_L2_READING': {
        'name': 'Proportion of children at the end of lower secondary education reaching minimum proficiency in reading', 
        'compare': ['EDUNF_ROFST_L2', 'EDU_SDG_STU_L2_MATH', 'EDU_SDG_STU_L1_GLAST_READING', 'EDU_SDG_STU_L1_G2OR3_READING', 'EDUNF_NERA_L2']
    },
    'EDU_SDG_STU_L1_GLAST_MATH': {
        'name': 'Proportion of children at the end of primary education reaching minimum proficiency in math',
        'compare': ['EDU_SDG_STU_L1_GLAST_READING', 'EDUNF_NERA_L1_PT']
    }
}
data = pd.DataFrame()
inds = set()
for key,value in indicators_dict.items():
    inds.add(key)
    inds.update(value['compare'])
    

for ind in inds:
    sdmx = pd.read_csv(sdmx_url.format(ind))
    sdmx['CODE'] = ind        
    data = data.append(sdmx)

In [6]:
# Create controls
county_options = [
    {"label": str(country), "value": str(country)} for country in data['Geographic area'].unique()
]

In [16]:
county_options

[{'label': 'Azerbaijan', 'value': 'Azerbaijan'},
 {'label': 'Bulgaria', 'value': 'Bulgaria'},
 {'label': 'Croatia', 'value': 'Croatia'},
 {'label': 'Hungary', 'value': 'Hungary'},
 {'label': 'Kyrgyzstan', 'value': 'Kyrgyzstan'},
 {'label': 'Kazakhstan', 'value': 'Kazakhstan'},
 {'label': 'Lithuania', 'value': 'Lithuania'},
 {'label': 'Republic of Moldova', 'value': 'Republic of Moldova'},
 {'label': 'Russian Federation', 'value': 'Russian Federation'},
 {'label': 'Slovakia', 'value': 'Slovakia'},
 {'label': 'Turkey', 'value': 'Turkey'},
 {'label': 'Czechia', 'value': 'Czechia'},
 {'label': 'Georgia', 'value': 'Georgia'},
 {'label': 'Latvia', 'value': 'Latvia'},
 {'label': 'Poland', 'value': 'Poland'},
 {'label': 'Romania', 'value': 'Romania'},
 {'label': 'Slovenia', 'value': 'Slovenia'},
 {'label': 'Albania', 'value': 'Albania'},
 {'label': 'Bosnia and Herzegovina', 'value': 'Bosnia and Herzegovina'},
 {'label': 'Belarus', 'value': 'Belarus'},
 {'label': 'Estonia', 'value': 'Estonia'},

In [17]:
years =[i for i in range(2010, 2020)]

indicators = data['Indicator'].unique()

{index: str(year) for index,year in enumerate(years)}

{0: '2010',
 1: '2011',
 2: '2012',
 3: '2013',
 4: '2014',
 5: '2015',
 6: '2016',
 7: '2017',
 8: '2018',
 9: '2019'}

In [22]:
data[data['CODE'] == 'EDU_SDG_STU_L2_READING']

# from mapbox import Geocoder

# mapbox_access_token = "pk.eyJ1IjoiamNyYW53ZWxsd2FyZCIsImEiOiJja2NkMW02aXcwYTl5MnFwbjdtdDB0M3oyIn0.zkIzPc4NSjLZvrY-DWrlZg"

# geocoder = Geocoder(access_token=mapbox_access_token)
# def geocode_address(address):
#     """Geocode street address into lat/long."""
#     response = geocoder.forward(address)
#     coords = response.json()['features'][0]['center']
#     # coords = coords.replace(']', '')
#     # coords = coords.replace('[', '')
#     coords

# data.apply(lambda row: pd.Series(geocode_address(row['Geographic area'])), axis=1)


,Geographic area,Indicator,Sex,Age,Residence,Wealth Quintile,TIME_PERIOD,OBS_VALUE,Unit of measure,OBS_FOOTNOTE,Frequency,DATA_SOURCE,Unit multiplier,OBS_STATUS,CODE
0,Albania,Proportion of children at the end of lower sec...,Total,Total,Total,Total,2000,29.64437,%,NaN,Annual,UIS,Units,A,EDU_SDG_STU_L2_READING
1,Albania,Proportion of children at the end of lower sec...,Total,Total,Total,Total,2009,43.33176,%,NaN,Annual,UIS,Units,A,EDU_SDG_STU_L2_READING
2,Albania,Proportion of children at the end of lower sec...,Total,Total,Total,Total,2012,47.66456,%,NaN,Annual,UIS,Units,A,EDU_SDG_STU_L2_READING
3,Albania,Proportion of children at the end of lower sec...,Total,Total,Total,Total,2015,49.72273,%,NaN,Annual,UIS,Units,A,EDU_SDG_STU_L2_READING
4,Albania,Proportion of children at the end of lower sec...,Total,Total,Total,Total,2018,47.76432,%,NaN,Annual,UIS,Units,A,EDU_SDG_STU_L2_READING
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
318,Turkey,Proportion of children at the end of lower sec...,Male,Total,Total,Total,2009,66.56000,%,NaN,Annual,UIS,Units,A,EDU_SDG_STU_L2_READING
319,Turkey,Proportion of children at the end of lower sec...,Male,Total,Total,Total,2012,69.10000,%,NaN,Annual,UIS,Units,A,EDU_SDG_STU_L2_READING
320,Turkey,Proportion of children at the end of lower sec...,Male,Total,Total,Total,2015,53.92574,%,NaN,Annual,UIS,Units,A,EDU_SDG_STU_L2_READING
321,Turkey,Proportion of children at the end of lower sec...,Male,Total,Total,Total,2018,68.42978,%,NaN,Annual,UIS,Units,A,EDU_SDG_STU_L2_READING
